# Vermont Geo-Data

* Q1: How much of our trail is on our land?

Data Sources:

https://geodata.vermont.gov/datasets/09cf47e1cf82465e99164762a04f3ce6_0/explore?location=43.863030%2C-73.290290%2C13.73


In [32]:
import os

import pandas as pd
import numpy as np

import geopandas as gpd

import matplotlib.pyplot as plt
import folium
from folium import GeoJson

import pyproj
import gpxpy



from xlwings import view

In [33]:
path_data = os.path.join('..', 'data')
path_img = os.path.join('..', 'img')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

In [34]:
# Load the GeoJSON file - should take ~30s

gdf = gpd.read_file(os.path.join(path_data, 'FS_VCGI_OPENDATA_Cadastral_VTPARCELS_poly_standardized_parcels_SP_v1_1924587466530394265.geojson'))

In [35]:
# Load GPX

gpx_file = open(os.path.join(path_data, 'Getting_a_trace_of_the_backyard_trail.gpx'), 'r')
gpx = gpxpy.parse(gpx_file)

In [36]:
# Display the first few rows
print(gdf.head())

# Get column names
print(gdf.columns)

# Get basic information about the dataset
print(gdf.info())

   OBJECTID           SPAN     GLIST_SPAN       MAPID      PARCID PROPTYPE  \
0         1  039-012-11586  039-012-11586  025/180.00  025/180.00   PARCEL   
1         2  039-012-11587  039-012-11587  034/031.00  034/031.00   PARCEL   
2         3  039-012-11001  039-012-11588  026/015.00  226/015.01   PARCEL   
3         4  039-012-11589  039-012-11589  006/063.01  006/063.01   PARCEL   
4         5  039-012-11590  039-012-11590  025/125.00  025/125.00   PARCEL   

   YEAR  GLYEAR        TOWN       TNAME SOURCENAME SOURCETYPE SOURCEDATE  \
0  2024  2023.0  BARRE TOWN  Barre Town       None    UNKNOWN    UNKNOWN   
1  2024  2023.0  BARRE TOWN  Barre Town       None    UNKNOWN    UNKNOWN   
2  2024  2023.0  BARRE TOWN  Barre Town       None    UNKNOWN    UNKNOWN   
3  2024  2023.0  BARRE TOWN  Barre Town       None    UNKNOWN    UNKNOWN   
4  2024  2023.0  BARRE TOWN  Barre Town       None    UNKNOWN    UNKNOWN   

  EDITMETHOD   EDITOR EDITDATE MATCHSTAT EDITNOTE  \
0    UNKNOWN  UNKNOWN

In [37]:
# Use this to locate parcel IDs

flds = ['OBJECTID', 'OWNER1', 'OWNER2', 'ADDRGL1', 'ADDRGL2', 'E911ADDR', 'TOWN']

my_parcel = gdf[gdf['OWNER1'].fillna('').str.contains('Crounse', case=False)]
print(my_parcel[flds])

other_parcels = gdf[(gdf['E911ADDR'].fillna('').str.contains('BROWN')) &
                    (gdf['TOWN'].fillna('').str.contains('SHOREHAM'))]
print(other_parcels[flds])

        OBJECTID           OWNER1            OWNER2            ADDRGL1  \
170787    170788    CROUNSE BRIAN  POOLER MADELEINE  49 INDIAN PIPE LN   
245455    245456  CROUNSE DAVID C    MARTINY LISA C       37 HARVEY RD   

       ADDRGL2      E911ADDR       TOWN  
170787    None   97 BROWN RD   SHOREHAM  
245455    None  37 HARVEY RD  UNDERHILL  
        OBJECTID                                  OWNER1  \
170718    170719   O'NEILL/KNAPP TRUSTEES. O'NEILL TRUST   
170787    170788                           CROUNSE BRIAN   
170789    170790                         MULLIGAN GEORGE   
170807    170808                      MATECKI KIMBERLY S   
170811    170812                          STOCKLEN AMY C   
170816    170817                            WOOD LOREN C   
170823    170824                            HOTTE STEVEN   
170824    170825  ORTIZ VALERIE & GRIFFIN KEVIN TRUSTEES   
170928    170929                            BOIRE ERIC E   
170936    170937                    CEMETERY AMES-W

In [38]:
#view(other_parcels)

In [39]:
ids = [170788, 170947]

parcels = gdf[gdf['OBJECTID'].isin(ids)].copy()

In [40]:
parcels.set_crs(epsg=32145, inplace=True, allow_override=True)
parcels = parcels.to_crs(epsg=4326)


In [41]:
# Extract track points
track_points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            track_points.append([point.latitude, point.longitude])


In [42]:
#parcels = parcels.to_crs(epsg=4326)

# Create a projection for accurate centroid calculation
# Here we use a Lambert Azimuthal Equal Area projection centered on Vermont
vermont_laea = pyproj.Proj(proj='laea', lat_0=44, lon_0=-72.7, datum='WGS84')

# Project to LAEA, calculate centroid, then project back to WGS84
parcels_proj = parcels.to_crs(vermont_laea.crs)
centroid_proj = parcels_proj.geometry.centroid
centroid_wgs84 = pyproj.transform(vermont_laea, pyproj.Proj('EPSG:4326'), 
                                  centroid_proj.x.mean(), centroid_proj.y.mean())

# Create map centered on the calculated centroid
m = folium.Map(location=[44, -72.7], zoom_start=10)

# Add parcels to the map
GeoJson(parcels_proj).add_to(m)


/var/folders/0y/0zmp_q2d027bgwpkl0ykqpyc0000gn/T/ipykernel_10236/2114561091.py:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  centroid_wgs84 = pyproj.transform(vermont_laea, pyproj.Proj('EPSG:4326'),


In [43]:
# Create a map
m = folium.Map()

# Add parcels to the map
folium.GeoJson(
    parcels,
    style_function=lambda feature: {
        'fillColor': 'orange',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7,
    }
).add_to(m)

# Add GPX track to the map
folium.PolyLine(track_points, color="red", weight=2.5, opacity=1).add_to(m)

# Fit the map to include both parcels and GPX track
bounds = parcels.total_bounds
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Save the map
m.save(os.path.join(path_img, "parcels_and_gpx_map.html"))


In [44]:
m

In [45]:
# Ensure the GeoDataFrame is in a suitable projection
#parcels = parcels.to_crs(epsg=4326)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the parcels
parcels.plot(ax=ax, edgecolor='black', facecolor='lightblue', alpha=0.5)

# Remove axis
ax.axis('off')

# Add a title
ax.set_title('Parcel Map')

# Save the figure
fig.savefig(os.path.join(path_img, 'parcel_map_v1.png'), dpi=300, bbox_inches='tight')

# Display the plot (this will work in VS Code)
fig.show()

/var/folders/0y/0zmp_q2d027bgwpkl0ykqpyc0000gn/T/ipykernel_10236/4268232053.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


In [46]:

track_points = np.array(track_points)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the parcels
parcels.plot(ax=ax, edgecolor='black', facecolor='lightblue', alpha=0.5)

# Plot the GPX track
#ax.plot(track_points[:, 0], track_points[:, 1], color='red', linewidth=2, alpha=0.7)

# Remove axis
ax.axis('off')

# Add a title
plt.title('Parcels and GPX Track')

# Adjust the plot limits to include both parcels and GPX track
# x_min = min(parcels.total_bounds[0], track_points[:, 0].min())
# x_max = max(parcels.total_bounds[2], track_points[:, 0].max())
# y_min = min(parcels.total_bounds[1], track_points[:, 1].min())
# y_max = max(parcels.total_bounds[3], track_points[:, 1].max())

# ax.set_xlim(x_min, x_max)
# ax.set_ylim(y_min, y_max)

# Save the figure
plt.savefig(os.path.join(path_img, 'map_parcels_v2.png'), dpi=300, bbox_inches='tight')

# Try to display the plot
try:
    plt.show()
except:
    print("Unable to display the plot interactively. Please open 'parcels_and_gpx_map.png' to view the map.")

print("Map saved.")

Map saved.


/var/folders/0y/0zmp_q2d027bgwpkl0ykqpyc0000gn/T/ipykernel_10236/458744799.py:32: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [47]:

track_points = np.array(track_points)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the parcels
parcels.plot(ax=ax, edgecolor='black', facecolor='lightblue', alpha=0.5)

# Plot the GPX track
ax.plot(track_points[:, 1], track_points[:, 0], color='red', linewidth=2, alpha=0.7)

# Remove axis
ax.axis('off')

# Add a title
plt.title('Parcels and GPX Track')

# Adjust the plot limits to include both parcels and GPX track
x_min = min(parcels.total_bounds[0], track_points[:, 1].min())
x_max = max(parcels.total_bounds[2], track_points[:, 1].max())
y_min = min(parcels.total_bounds[1], track_points[:, 0].min())
y_max = max(parcels.total_bounds[3], track_points[:, 0].max())

# x_min = min(track_points[:, 0])
# x_max = max(track_points[:, 0])
# y_min = min(track_points[:, 1])
# y_max = max(track_points[:, 1])

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

# Save the figure
plt.savefig(os.path.join(path_img, 'map_gpx_and_parcels_v3.png'), dpi=300, bbox_inches='tight')

# Try to display the plot
try:
    plt.show()
except:
    print("Unable to display the plot interactively. Please open 'parcels_and_gpx_map.png' to view the map.")

print("Map saved.")

Map saved.


/var/folders/0y/0zmp_q2d027bgwpkl0ykqpyc0000gn/T/ipykernel_10236/421773908.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
